### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

### QUERIES

In [244]:
q_seg = '''WITH users_table AS (    
    SELECT FORMAT_DATE('%Y-%m',god.order_date) AS month,
        god.user_id AS user,
        gec.segment AS segment,
        SUM(god.qty_orders) AS orders  
    FROM `peya-delivery-and-support.automated_tables_reports.glovo_orders_date` AS god
    LEFT JOIN `peya-growth-and-onboarding.automated_tables_reports.glovo_user_segmentation_ec` AS gec ON god.user_id = gec.user_id
    WHERE god.country_code = 'EC'
        AND god.order_status = 'CONFIRMED'
        AND god.order_date BETWEEN DATE('2020-12-01') AND DATE('2021-02-28') 
    GROUP BY 1,2,3)
SELECT ut.month AS Month,
       IFNULL(ut.segment,'-') AS Segment,
       CASE WHEN UPPER(ut.segment) LIKE '%%LOYAL%%' THEN 'Loyal' ELSE 'No Loyal' END AS Loyalty,
       CASE WHEN ut.orders >= 7 THEN '+7' ELSE '-7' END AS Tipo,
       COUNT(DISTINCT ut.user) AS Users,
       SUM(ut.orders) AS Orders
FROM users_table AS ut
GROUP BY 1,2,3,4'''

q_ord = '''WITH users_table AS (    
    SELECT FORMAT_DATE('%Y-%m',god.order_date) AS month,
        god.user_id AS user,
        SUM(god.qty_orders) AS orders  
    FROM `peya-delivery-and-support.automated_tables_reports.glovo_orders_date` AS god
    WHERE god.country_code = 'EC'
        AND god.order_status = 'CONFIRMED'
        AND god.order_date BETWEEN DATE('2021-02-01') AND DATE('2021-02-28') 
    GROUP BY 1,2)
SELECT FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       IFNULL(gec.segment,'-') AS Segment,
       CASE WHEN UPPER(gec.segment) LIKE '%%LOYAL%%' THEN 'Loyal' ELSE 'No Loyal' END AS Loyalty,
       CASE WHEN ut.orders >= 7 THEN '+7' ELSE '-7' END AS Tipo,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.user.id ELSE NULL END) AS Users,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Orders
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
LEFT JOIN `peya-growth-and-onboarding.automated_tables_reports.glovo_user_segmentation_ec` AS gec ON o.user.id = gec.user_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
LEFT JOIN users_table AS ut ON o.user.id = ut.user
WHERE o.registered_date >= DATE('2021-03-01')
      AND cn.country_name = 'Ecuador'
GROUP BY 1,2,3,4
ORDER BY 1 ASC'''

q_t = '''WITH users_table AS (    
    SELECT FORMAT_DATE('%Y-%m',o.registered_date) AS month,
        o.user.id AS user,
        COUNT(DISTINCT o.order_id) AS orders
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o 
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    WHERE cn.country_name = 'Ecuador'
        AND o.order_status = 'CONFIRMED'
        AND o.registered_date BETWEEN DATE('2021-03-01') AND DATE('2021-06-30') 
    GROUP BY 1,2)
SELECT ut.month AS Month,
       CASE WHEN ut.orders >= 1 AND ut.orders <= 3 THEN '1-3' 
            WHEN ut.orders >= 4 AND ut.orders <= 6 THEN '4-6'
            WHEN ut.orders >=7 THEN '+7' ELSE '-' END AS Tipo,
       COUNT(DISTINCT ut.user) AS Users,
       SUM(ut.orders) AS Orders,
       SUM(ut.orders) /  COUNT(DISTINCT ut.user) AS Freq
FROM users_table AS ut
GROUP BY 1,2
ORDER BY 1 ASC'''

In [136]:
# Descargo la data
bq_seg = pd.io.gbq.read_gbq(q_seg, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 92.94rows/s]


In [188]:
bq_ord = pd.io.gbq.read_gbq(q_ord, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████████| 90/90 [00:00<00:00, 232.37rows/s]


In [245]:
bq_t = pd.io.gbq.read_gbq(q_t, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 32.59rows/s]


In [246]:
# Copio las bases
seg = bq_seg.copy()
orders = bq_ord.copy()
t = bq_t.copy()

### TRABAJO

In [247]:
# Trabajo la segmentacion de base
pts_t = []
for i in t.sort_values(by=['Month'],ascending=True)['Month'].unique():
    pts_t.append(t[t['Month'] == i].copy())
for i in pts_t:
    i['%Orders'] = i['Orders'] / i['Orders'].sum()
    i['%Users'] = i['Users'] / i['Users'].sum()
    i['Freq'] = i['Orders'] / i['Users']

In [260]:
pts_t[3]

,Month,Tipo,Users,Orders,Freq,%Orders,%Users
9,2021-06,1-3,1825928,2978575,1.631266,0.351961,0.716122
10,2021-06,4-6,419985,1996517,4.753782,0.235917,0.164717
11,2021-06,+7,303830,3487706,11.479136,0.412122,0.119161


In [199]:
# Hago una PT para explicar la fuerza del grupo de usuarios
index = ['Month','Loyalty','Tipo']
values = ['Users','Orders']
pt_seg = seg.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()
pts_seg = []
for i in seg.sort_values(by=['Month'],ascending=True)['Month'].unique():
    pts_seg.append(pt_seg[pt_seg['Month'] == i].copy())
for i in pts_seg:
    i['%Orders'] = i['Orders'] / i['Orders'].sum()
    i['%Users'] = i['Users'] / i['Users'].sum()
    i['Freq'] = i['Orders'] / i['Users']

In [200]:
pts_seg[0]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
0,2020-12,Loyal,+7,178636,15547,0.382918,0.104340,11.490062
1,2020-12,Loyal,-7,154460,50826,0.331095,0.341107,3.038996
2,2020-12,No Loyal,+7,9421,311,0.020195,0.002087,30.292605
3,2020-12,No Loyal,-7,123996,82319,0.265793,0.552465,1.506287


In [201]:
pts_seg[0]['Orders'].sum(),pts_seg[0]['Users'].sum(),pts_seg[0]['Orders'].sum()/pts_seg[0]['Users'].sum()

(466513, 149003, 3.1308966933551674)

In [202]:
pts_seg[1]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
4,2021-01,Loyal,+7,170763,14721,0.367578,0.096949,11.599959
5,2021-01,Loyal,-7,143693,48339,0.309308,0.318349,2.972610
6,2021-01,No Loyal,+7,10538,475,0.022684,0.003128,22.185263
7,2021-01,No Loyal,-7,139569,88308,0.300431,0.581574,1.580480


In [203]:
pts_seg[1]['Orders'].sum(),pts_seg[1]['Users'].sum(),pts_seg[1]['Orders'].sum()/pts_seg[1]['Users'].sum()

(464563, 151843, 3.0594956632837866)

In [204]:
pts_seg[2]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
8,2021-02,Loyal,+7,132474,11897,0.319909,0.086132,11.135076
9,2021-02,Loyal,-7,129609,46314,0.312990,0.335303,2.798484
10,2021-02,No Loyal,+7,20296,1014,0.049012,0.007341,20.015779
11,2021-02,No Loyal,-7,131720,78901,0.318088,0.571225,1.669434


In [205]:
pts_seg[2]['Orders'].sum(),pts_seg[2]['Users'].sum(),pts_seg[2]['Orders'].sum()/pts_seg[2]['Users'].sum()

(414099, 138126, 2.9979801051214108)

In [206]:
# Hago una PT para explicar la fuerza del grupo de usuarios actualmente
index = ['Month','Loyalty','Tipo']
values = ['Users','Orders']
pt_ord = orders.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()
pts = []
for i in pt_ord.sort_values(by=['Month'],ascending=True)['Month'].unique():
    pts.append(pt_ord[pt_ord['Month'] == i].copy())
for i in pts:
    i['%Orders'] = i['Orders'] / i['Orders'].sum()
    i['%Users'] = i['Users'] / i['Users'].sum()
    i['Freq'] = i['Orders'] / i['Users']

In [207]:
pts[0]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
0,2021-03,Loyal,+7,20231,6275,0.124073,0.060633,3.224064
1,2021-03,Loyal,-7,38819,22123,0.238070,0.213765,1.754690
2,2021-03,No Loyal,+7,1085,408,0.006654,0.003942,2.659314
3,2021-03,No Loyal,-7,102922,74686,0.631203,0.721660,1.378063


In [208]:
pts[0]['Orders'].sum(),pts[0]['Users'].sum(),pts[0]['Orders'].sum()/pts[0]['Users'].sum()

(163057, 103492, 1.5755517334673212)

In [209]:
pts[1]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
4,2021-04,Loyal,+7,36798,6536,0.099594,0.038167,5.630049
5,2021-04,Loyal,-7,80655,29087,0.218294,0.169852,2.772888
6,2021-04,No Loyal,+7,1756,422,0.004753,0.002464,4.161137
7,2021-04,No Loyal,-7,250270,135204,0.677359,0.789517,1.851055


In [210]:
pts[1]['Orders'].sum(),pts[1]['Users'].sum(),pts[1]['Orders'].sum()/pts[1]['Users'].sum()

(369479, 171249, 2.1575542046960856)

In [211]:
pts[2]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
8,2021-05,Loyal,+7,35594,5947,0.085850,0.032745,5.985203
9,2021-05,Loyal,-7,81917,26475,0.197577,0.145774,3.094127
10,2021-05,No Loyal,+7,1675,367,0.004040,0.002021,4.564033
11,2021-05,No Loyal,-7,295421,148828,0.712533,0.819461,1.984983


In [212]:
pts[2]['Orders'].sum(),pts[2]['Users'].sum(),pts[2]['Orders'].sum()/pts[2]['Users'].sum()

(414607, 181617, 2.2828644895576957)

In [213]:
pts[3]

,Month,Loyalty,Tipo,Orders,Users,%Orders,%Users,Freq
12,2021-06,Loyal,+7,28285,5344,0.089329,0.037052,5.292852
13,2021-06,Loyal,-7,62551,22482,0.197547,0.155876,2.782270
14,2021-06,No Loyal,+7,1297,317,0.004096,0.002198,4.091483
15,2021-06,No Loyal,-7,224505,116087,0.709027,0.804874,1.933937


In [214]:
pts[3]['Orders'].sum(),pts[3]['Users'].sum(),pts[3]['Orders'].sum()/pts[3]['Users'].sum()

(316638, 144230, 2.195368508632046)